# Assignment 1
# Segmenting and Clustering Neighborhoods in Toronto

In this assignment, we will explore and cluster the neighborhoods in Toronto using simple pandas code to scrap a [Wikipedia page](https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M). The Wikipedia page comprises three tables, and we analyze the first table that contains the Postal Code,  Borough, and Neighbourhood.


### Import necessary Libraries

In [16]:
import pandas as pd # library for data analsysis

### Scrap the neighborhoods table

In [17]:
# Define the URL for the source data
url =  'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

# Read HTML tables into a list of DataFrame objects.
df = pd.read_html(url)

# Get the number of tables scrapped from the Wikipedia page
print(len(df))

3


In [19]:
# Select the appropriate neighborhoods table
df = df[0]
df.head()

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### Cleaning and arranging the dataset. 

As we can see on the above table's first and second row, the DataFrame comprises multiple unassigned boroughs and neighborhoods. Hence, we exclude boroughs that are not assigned from the DataFrame. 

In [24]:
# Exclude unassigned boroughs and reset the index of the DataFrame
df = df[df['Borough']!='Not assigned'].reset_index(drop=True)

# Rename the first column
df = df.rename(columns={"Postal Code": "PostalCode"})

df.head(15)


,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


### Get Shape of Pandas DataFrame

In [22]:
df.shape

(103, 3)